# Mount drive

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Install Dependencies

In [2]:
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 186 kB 8.3 MB/s 
     |████████████████████████████████| 243 kB 48.6 MB/s 
     |████████████████████████████████| 118 kB 55.1 MB/s 
     |████████████████████████████████| 2.1 MB 7.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.6 MB/s 
     |████████████████████████████████| 895 kB 59.8 MB/s 
     |████████████████████████████████| 1.9 MB 7.2 MB/s 
     |████████████████████████████████| 50 kB 4.1 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149868 sha256=bf0b8bd9c4ce2ffa7fd6cd1310554d5383685add02fef040c87ffa1c41a78d99
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


# import Libraries

In [59]:
#import libraries

import numpy as np
import librosa as lb
import os
import pandas as pd
import torch
from datasets import load_dataset, load_metric
import json
import IPython.display as ipd
from sklearn.model_selection import train_test_split

# sample audio file

In [4]:
audio = lb.load("/content/gdrive/MyDrive/jf8398hf30f0381738rucj3828chfdnchs.tar.gz (Unzipped Files)/fluent_speech_commands_dataset/wavs/speakers/zZezMeg5XvcbRdg3/ca60c080-45e0-11e9-81ce-69b74fd7e64e.wav")

#sample audio file
print(audio)
ipd.Audio(data = audio[0], rate = 16000, autoplay = True)   #play audio file

(array([0.        , 0.        , 0.        , ..., 0.00135181, 0.00201907,
       0.        ], dtype=float32), 22050)


# load training csv file

In [5]:
train_df = pd.read_csv("/content/gdrive/MyDrive/jf8398hf30f0381738rucj3828chfdnchs.tar.gz (Unzipped Files)/fluent_speech_commands_dataset/data/train_data.csv")
train_df.sample(5)

,Unnamed: 0,path,speakerId,transcription,action,object,location
21807,21807,wavs/speakers/zaEBPeMY4NUbDnZy/1195acf0-44d7-1...,zaEBPeMY4NUbDnZy,Fetch my socks,bring,socks,none
16153,16153,wavs/speakers/R3mXwwoaX9IoRVKe/af4cd8a0-4550-1...,R3mXwwoaX9IoRVKe,Quieter,decrease,volume,none
11178,11178,wavs/speakers/LR5vdbQgp3tlMBzB/6d6e3770-45ce-1...,LR5vdbQgp3tlMBzB,Start the music,activate,music,none
20740,20740,wavs/speakers/xwzgmmv5ZOiVaxXz/ad8bc9b0-453a-1...,xwzgmmv5ZOiVaxXz,Washroom lights on,activate,lights,washroom
7277,7277,wavs/speakers/EExgNZ9dvgTE3928/d6f40f80-4614-1...,EExgNZ9dvgTE3928,Bring my shoes,bring,shoes,none


# Sample testing

In [6]:
#train

audio_path = []
transcription = []
initial_audio = "/content/gdrive/MyDrive/jf8398hf30f0381738rucj3828chfdnchs.tar.gz (Unzipped Files)/fluent_speech_commands_dataset"
#write json file

with open("/content/sample_data/train.json" , "w") as f:

  for i, data in train_df.iterrows():
    row_json = {"file" :initial_audio + "/" + data["path"], "text" : data["transcription"]}
    json.dump(row_json, f)
    f.write("\n")
    if i == 5000:
      break
    

# load testing csv file

In [7]:
test_df = pd.read_csv("/content/gdrive/MyDrive/jf8398hf30f0381738rucj3828chfdnchs.tar.gz (Unzipped Files)/fluent_speech_commands_dataset/data/test_data.csv")
test_df.sample(100)

,Unnamed: 0,path,speakerId,transcription,action,object,location
482,482,wavs/speakers/7B4XmNppyrCK977p/9aa34d80-45c8-1...,7B4XmNppyrCK977p,Turn off the lights,deactivate,lights,none
1483,1483,wavs/speakers/k5bqyxx2lzIbrlg9/fa339150-4529-1...,k5bqyxx2lzIbrlg9,Decrease the temperature in the kitchen,decrease,heat,kitchen
740,740,wavs/speakers/7B4XmNppyrCK977p/0a0d0b30-45cd-1...,7B4XmNppyrCK977p,Turn up the volume,increase,volume,none
492,492,wavs/speakers/7B4XmNppyrCK977p/b6b82d10-45c8-1...,7B4XmNppyrCK977p,Lights off,deactivate,lights,none
1949,1949,wavs/speakers/oOK5kxoW7dskMbaK/de9ccbf0-44e8-1...,oOK5kxoW7dskMbaK,Play,activate,music,none
...,...,...,...,...,...,...,...
3044,3044,wavs/speakers/V4ejqNL4xbUKkYrV/f5d94630-459d-1...,V4ejqNL4xbUKkYrV,Set the language,change language,none,none
3677,3677,wavs/speakers/V4ZbwLm9G5irobWn/e06dd1e0-4529-1...,V4ZbwLm9G5irobWn,Increase the heating in the kitchen,increase,heat,kitchen
2318,2318,wavs/speakers/oOK5kxoW7dskMbaK/995e68f0-44ed-1...,oOK5kxoW7dskMbaK,Stop music,deactivate,music,none
322,322,wavs/speakers/4BrX8aDqK2cLZRYl/2af7d5b0-4531-1...,4BrX8aDqK2cLZRYl,Bedroom lights off,deactivate,lights,bedroom


In [8]:
# test

audio_path = []
transcription = []
initial_audio = "/content/gdrive/MyDrive/jf8398hf30f0381738rucj3828chfdnchs.tar.gz (Unzipped Files)/fluent_speech_commands_dataset"
#write json file

with open("/content/sample_data/test.json" , "w") as f:

  for i, data in test_df.iterrows():
    row_json = {"file" :initial_audio + "/" + data["path"], "text" : data["transcription"]}
    json.dump(row_json, f)
    f.write("\n")
    if (i == 51):
      break

# prepare dataset

In [9]:
timit = load_dataset("json", 
                     data_files = {"train" : "/content/sample_data/train.json"}
                     )


Using custom data configuration default-2bec19970d3c18a2


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2bec19970d3c18a2/0.0.0/dc7ee63ec8b554c48ecc5a8a6fbe27af8071408c244e4347cf9222d6206d83a2. Subsequent calls will reuse this data.


In [10]:
timit = timit["train"].train_test_split(0.1)

In [11]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'text'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['file', 'text'],
        num_rows: 501
    })
})

# Trial 

In [12]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(timit["train"].remove_columns(["file"]), num_examples=20)

,text
0,Lights on
1,Lights off in the bedroom
2,Turn up the heat in the bathroom
3,Bring socks
4,Turn up the bedroom heat
5,Increase the temperature in the bathroom
6,Turn the heat down
7,Washroom heat down
8,"I couldn't hear anything, turn up the volume"
9,Go get me some juice


In [14]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\'’"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [15]:
timit = timit.map(remove_special_characters)

In [16]:
show_random_elements(timit["train"].remove_columns(["file"]))

,text
0,turn the volume down
1,make it louder
2,decrease the temperature in the kitchen
3,turn the bedroom heat down
4,lights on in the bedroom
5,turn off the lamp
6,kitchen lights off
7,could you increase the heating please
8,turn on the lights in the bedroom
9,reduce audio volume


In [17]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [18]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

In [19]:
vocab_list = list(set(vocabs["train"]["vocab"][0]))

In [31]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
#vocab_dict

In [21]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [22]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

28

In [24]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [25]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [26]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [41]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/new_123")

In [28]:
timit["train"][0]

{'file': '/content/gdrive/MyDrive/jf8398hf30f0381738rucj3828chfdnchs.tar.gz (Unzipped Files)/fluent_speech_commands_dataset/wavs/speakers/AY5e3mMgZkIyG3Ox/78a36910-446d-11e9-beaa-1f3e1eab7bff.wav',
 'text': 'could you increase the heating '}

In [29]:
import soundfile as sf

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = lb.load(batch["file"], sr = 16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [49]:
timit = timit.map(speech_file_to_array_fn, remove_columns=timit.column_names["train"], num_proc=4)

In [50]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

In [51]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["speech"]).shape)
print("Sampling rate:", timit["train"][rand_int]["sampling_rate"])

Target text: stop 
Input array shape: (21845,)
Sampling rate: 16000


In [52]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [53]:
timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [34]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir = "/content/gdrive/MyDrive/new_123",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
timit_prepared["train"]

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 4500
})

In [ ]:
timit_prepared["test"]

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 501
})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

# inference

In [54]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [55]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [63]:
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/new_123/checkpoint-7000")

In [64]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to("cuda")

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  
  return batch

In [65]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt",
  ).input_values.to("cuda")

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]

  return batch

results = timit["test"].map(map_to_result)

In [66]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

NameError: ignored

In [ ]:
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

,pred_str,target_text
0,turn the lamp off,turn the lamp off
1,turn down the volume,turn down the volume
2,turn the bedroom lights on,turn the bedroom lights on
3,change language,change language
4,lights on in the kitchen,lights on in the kitchen
5,play,play
6,switch on the lamp,switch on the lamp
7,switch on the lights,switch on the lights
8,turn the temperature down,turn the temperature down
9,volume down,volume down
